In [ ]:
# --- Cell 0: Load Source Data & Auto-Detect Parameters ---
%matplotlib widget

import importlib
import sys
import traceback
import re
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipyfilechooser import FileChooser
from IPython.display import clear_output, display

import opym
import opym.metadata  # Ensure metadata module is available

# Reload modules
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.petakit)
    importlib.reload(opym.utils)
    importlib.reload(opym.viewer)
    importlib.reload(opym.metadata) # Added metadata reload
    importlib.reload(opym)
    print("Reloaded opym library.")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Create Interactive Widgets ---
print("\n1. Select your source data directory (OPM or LLSM).")
print("2. Choose the channel to use for the preview MIP.")
print("3. Click 'Load Data'.")

dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Source TIFF Directory (OPM or LLSM):</b>",
    show_only_dirs=True,
)

preview_channel_widget = widgets.IntText(
    value=2,
    description="Preview Channel:",
    style={"description_width": "initial"},
)

load_button = widgets.Button(description="Load Data", button_style="primary")
load_output = widgets.Output()

# --- Global variables to hold data and parameters ---
source_processed_dir: Path | None = None
mip_original: np.ndarray | None = None
stack_original: np.ndarray | None = None
vmin_original: float = 0
vmax_original: float = 1
data_type_source: opym.MicroscopyDataType = "UNKNOWN"

# --- Globals for OPM data (used in Cell 1) ---
get_stack_source = None
t_min_source = 0
t_max_source = 0
c_min_source = 0
c_max_source = 0
base_name_source = ""

# --- Default physical params (Will be updated by auto-detection) ---
default_xy_size = 0.136
default_z_step = 1.0
default_angle = 31.8


def on_load_button_clicked(b):
    global source_processed_dir, mip_original, vmin_original, vmax_original
    global get_stack_source, t_min_source, t_max_source, c_min_source
    global c_max_source, base_name_source, data_type_source
    global stack_original
    # Allow updating defaults for Cell 1
    global default_z_step, default_angle, default_xy_size

    with load_output:
        clear_output(wait=True)
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected.")
            return

        source_processed_dir = Path(dir_chooser.value)
        preview_channel = preview_channel_widget.value

        try:
            # 1. Detect Data Type
            data_type_source = opym.detect_microscopy_data_type(source_processed_dir)

            if data_type_source == "OPM":
                print(f"--- Loading OPM data from: {source_processed_dir.name} ---")
                (
                    get_stack,
                    t_min, t_max,
                    c_min, c_max,
                    z_max, y, x,
                    base_name,
                ) = opym.load_tiff_series(source_processed_dir)

                # Store globals
                get_stack_source = get_stack
                t_min_source = t_min
                t_max_source = t_max
                c_min_source = c_min
                c_max_source = c_max
                base_name_source = base_name

                # Validate preview channel
                if not c_min <= preview_channel <= c_max:
                    print(f"⚠️ Warning: Channel {preview_channel} not in range ({c_min}-{c_max}). Defaulting to C={c_min}.")
                    preview_channel = c_min

                print(f"  Generating MIP from T={t_min}, C={preview_channel}...")
                stack_original = get_stack(t_min, preview_channel)
                mip_original = np.max(stack_original, axis=0)

            elif data_type_source == "LLSM":
                print(f"--- Loading LLSM preview from: {source_processed_dir.name} ---")
                (
                    get_stack,
                    t_min, t_max,
                    c_min, c_max,
                    z_max, y, x,
                    base_name,
                ) = opym.load_llsm_tiff_series(source_processed_dir)

                if not c_min <= preview_channel <= c_max:
                    print(f"⚠️ Warning: Channel {preview_channel} not in range ({c_min}-{c_max}). Defaulting to C={c_min}.")
                    preview_channel = c_min

                print(f"  Generating MIP from T={t_min}, C={preview_channel}...")
                stack_original = get_stack(t_min, preview_channel)
                mip_original = np.max(stack_original, axis=0)
                
                # Store globals
                get_stack_source = None
                base_name_source = base_name

            else:
                print("❌ ERROR: Unknown data type. Check directory contents.")
                return

            # 2. Calculate Display Levels
            if mip_original is not None:
                vmin_original, vmax_original = np.percentile(mip_original, [1, 99.9])
                if vmin_original >= vmax_original:
                    vmax_original = np.max(mip_original)

            # --- 3. AUTO-DETECT METADATA ---
            print("\n--- Auto-detecting Metadata ---")
            
            # A. Find Metadata File
            metadata_file = None
            if base_name_source:
                # OPM metadata usually in parent dir: base_name + "_metadata.txt"
                possible_meta = source_processed_dir.parent / (base_name_source + "_metadata.txt")
                if possible_meta.exists():
                    metadata_file = possible_meta
            
            # B. Parse Z-Step
            if metadata_file:
                try:
                    found_z = opym.metadata.parse_z_step(metadata_file, default_z_step=default_z_step)
                    if found_z:
                        default_z_step = found_z
                        print(f"✅ Found Z-Step in metadata: {default_z_step} µm")
                    else:
                        print(f"⚠️ Could not parse Z-step from {metadata_file.name}. Keeping default ({default_z_step} µm).")
                except Exception:
                    print(f"⚠️ Error reading metadata. Keeping default Z ({default_z_step} µm).")
            else:
                print(f"ℹ️ No metadata file found. Keeping default Z ({default_z_step} µm).")

            print(f"\n✅ {data_type_source} data loaded successfully.")
            print(f"   Base Name: {base_name_source}")
            print(f"   Defaults for next step -> Z: {default_z_step}, Angle: {default_angle}")
            print("You can now run Cell 1.")

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not load data: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)


load_button.on_click(on_load_button_clicked)
display(widgets.VBox([dir_chooser, preview_channel_widget, load_button, load_output]))

In [ ]:
# --- Cell 1: Interactive Parameter Tuner (Client-Server Version) ---

import ipywidgets as widgets
import numpy as np
from IPython.display import display, clear_output
import traceback
import matplotlib.pyplot as plt
import opym  # Ensure opym is imported
import sys

# --- Global to hold parameters for viewer cells ---
processed_viewer_params = None

# --- Create Parameter Sliders ---
# Note: defaults might have been updated by Cell 0
angle_slider = widgets.FloatSlider(
    value=default_angle,
    min=10.0,
    max=40.0,
    step=0.01,
    description="Sheet Angle:",
    readout_format=".2f",
    style={"description_width": "initial"},
)
z_step_slider = widgets.FloatSlider(
    value=default_z_step,
    min=0.05,
    max=5.0,
    step=0.05,
    description="Z-Step (um):",
    readout_format=".2f",
    style={"description_width": "initial"},
)
xy_size_slider = widgets.FloatSlider(
    value=default_xy_size,
    min=0.05,
    max=0.2,
    step=0.001,
    description="XY Pixel Size (um):",
    readout_format=".3f",
    style={"description_width": "initial"},
)

# --- Create Buttons and Output Areas ---
run_view_button = widgets.Button(
    description="Run & View Processed Stack", button_style="success"
)
log_output = widgets.Output(
    layout={"border": "1px solid black", "height": "200px", "overflow_y": "scroll"}
)
view_output = widgets.Output()


# --- Define Button Click Handler ---
def on_run_view_button_clicked_v2(b):
    global processed_viewer_params
    run_view_button.disabled = True

    with log_output:
        clear_output(wait=True)
    with view_output:
        clear_output(wait=True)

    try:
        # Check if globals from Cell 0 are loaded
        if (
            source_processed_dir is None
            or mip_original is None
            or stack_original is None
        ):
            with log_output:
                print("❌ ERROR: Source data not loaded. Please run Cell 0 first.")
            return

        if data_type_source == "UNKNOWN":
            with log_output:
                print("❌ ERROR: Data type is UNKNOWN. Please re-run Cell 0.")
            return

        # 1. Get all parameters from widgets
        angle = angle_slider.value
        z_step = z_step_slider.value
        xy_size = xy_size_slider.value

        with log_output:
            print(f"--- Processing {data_type_source} data ---")
            print(f"   Angle={angle}, Z-Step={z_step}, XY-Size={xy_size}")

        # 2. Create unique directory name for this run
        dsr_dir_name = f"DSR_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}"
        ds_dir_name = f"DS_a{angle:.2f}_z{z_step:.2f}_xy{xy_size:.3f}"

        # 3. Prepare Parameters
        processing_params = {
            "ds_dir_name": ds_dir_name,
            "dsr_dir_name": dsr_dir_name,
            "sheet_angle_deg": angle,
            "z_step_um": z_step,
            "xy_pixel_size": xy_size,
            "save_mip": True,
        }

        # 4. SUBMIT JOB AND WAIT
        # We use the new Client-Server model
        with log_output:
            print("⏳ Submitting job to MATLAB server...")
            
            job_id = None
            if data_type_source == "OPM":
                job_id = opym.run_petakit_processing(source_processed_dir, **processing_params)
            elif data_type_source == "LLSM":
                job_id = opym.run_llsm_petakit_processing(source_processed_dir, **processing_params)
            
            if job_id:
                # Block here until the background server finishes this job
                # This allows us to load the result immediately after
                print(f"   Ticket: {job_id}")
                opym.wait_for_job(job_id) 
            else:
                raise RuntimeError("Job submission returned no ID.")

        # 5. Load the processed stack
        output_path = source_processed_dir / dsr_dir_name
        with log_output:
            print(f"✅ Processing complete. Loading results from: {output_path.name}")

        (
            get_stack_proc,
            t_min_proc, t_max_proc,
            c_min_proc, c_max_proc,
            z_max_proc,
            y_proc, x_proc,
            bn_proc,
        ) = opym.load_tiff_series(output_path)

        processed_viewer_params = (
            get_stack_proc,
            t_max_proc,
            z_max_proc,
            c_max_proc,
            y_proc,
            x_proc,
        )
        
        # Handle preview channel mismatch
        preview_channel_to_load = preview_channel_widget.value
        if not c_min_proc <= preview_channel_to_load <= c_max_proc:
            with log_output:
                print(f"⚠️ Warning: Preview channel {preview_channel_to_load} not found in result.")
                print(f"   Defaulting to C={c_min_proc}.")
            preview_channel_to_load = c_min_proc

        with log_output:
            print(f"   Generating preview MIP (C={preview_channel_to_load})...")

        stack_processed = get_stack_proc(t_min_proc, preview_channel_to_load)

        # 6. Calculate Orthoviews
        mip_original_xy = mip_original
        mip_original_xz = np.max(stack_original, axis=1)
        mip_original_yz = np.max(stack_original, axis=2)

        mip_processed_xy = np.max(stack_processed, axis=0)
        mip_processed_xz = np.max(stack_processed, axis=1)
        mip_processed_yz = np.max(stack_processed, axis=2)

        vmin_proc, vmax_proc = np.percentile(mip_processed_xy, [1, 99.9])
        if vmin_proc >= vmax_proc:
            vmax_proc = np.max(mip_processed_xy)

        # 7. Display Plots
        with view_output:
            fig, axes = plt.subplots(2, 3, figsize=(15, 8))

            # Row 0: Original
            axes[0, 0].imshow(mip_original_xy, cmap="gray", vmin=vmin_original, vmax=vmax_original)
            axes[0, 0].set_title(f"Original XY")
            
            axes[0, 1].imshow(mip_original_xz, cmap="gray", vmin=vmin_original, vmax=vmax_original)
            axes[0, 1].set_title(f"Original XZ")
            axes[0, 1].set_aspect("auto")

            axes[0, 2].imshow(mip_original_yz, cmap="gray", vmin=vmin_original, vmax=vmax_original)
            axes[0, 2].set_title(f"Original YZ")
            axes[0, 2].set_aspect("auto")

            # Row 1: Processed
            axes[1, 0].imshow(mip_processed_xy, cmap="gray", vmin=vmin_proc, vmax=vmax_proc)
            axes[1, 0].set_title(f"Processed XY")

            axes[1, 1].imshow(mip_processed_xz, cmap="gray", vmin=vmin_proc, vmax=vmax_proc)
            axes[1, 1].set_title(f"Processed XZ")
            axes[1, 1].set_aspect("auto")

            axes[1, 2].imshow(mip_processed_yz, cmap="gray", vmin=vmin_proc, vmax=vmax_proc)
            axes[1, 2].set_title(f"Processed YZ")
            axes[1, 2].set_aspect("auto")

            for ax in axes.flatten():
                ax.axis("off")

            plt.tight_layout()
            plt.show()

            with log_output:
                print("\n✅ Orthoviews displayed.")
                print("You can now run Cell 2 or 3 to view the stack interactively.")

    except Exception as e:
        with log_output:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)
    finally:
        run_view_button.disabled = False


# --- Link button and display UI ---
run_view_button.on_click(on_run_view_button_clicked_v2)

param_box = widgets.VBox([angle_slider, z_step_slider, xy_size_slider])
display(widgets.VBox([param_box, run_view_button, log_output, view_output]))

In [ ]:
# --- Cell 2: Launch the composite viewer ---
# (Run this cell after running Cell 1)

try:
    if processed_viewer_params is None:
        raise NameError
    opym.composite_viewer(*processed_viewer_params)
except NameError:
    print(
        "❌ ERROR: 'processed_viewer_params' not found. Please run Cell 1 to process data first."
    )
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")